<a href="https://colab.research.google.com/github/Fliptoss/ML_stuff/blob/main/Binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [17]:
df = pd.read_csv('/content/train.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [18]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [19]:
df['family'] = df['SibSp'] + df['Parch']
df.drop(columns=['SibSp','Parch'],inplace=True)

In [20]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [21]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [23]:
X_train

,Age,Fare,family
331,45.5,28.5000,0
733,23.0,13.0000,0
382,32.0,7.9250,0
704,26.0,7.8542,1
813,6.0,31.2750,6
...,...,...,...
106,21.0,7.6500,0
270,NaN,31.0000,0
860,41.0,14.1083,2
435,14.0,120.0000,3


In [24]:
# Without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

0.6927374301675978

In [25]:
np.mean(cross_val_score(DecisionTreeClassifier(), X, y, cv=10, scoring='accuracy'))

np.float64(0.6565293383270911)

In [26]:
from sklearn.preprocessing import Binarizer

In [29]:
trf=ColumnTransformer([
    ('bin', Binarizer(copy=False), ['family'])
], remainder = 'passthrough')

In [30]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [31]:
X_train_trf

array([[  0.    ,  45.5   ,  28.5   ],
       [  0.    ,  23.    ,  13.    ],
       [  0.    ,  32.    ,   7.925 ],
       ...,
       [  1.    ,  41.    ,  14.1083],
       [  1.    ,  14.    , 120.    ],
       [  1.    ,  21.    ,  77.2875]])

In [32]:
pd.DataFrame(X_train_trf, columns=['family', 'Fare', 'Age'])

,family,Fare,Age
0,0.0,45.5,28.5000
1,0.0,23.0,13.0000
2,0.0,32.0,7.9250
3,1.0,26.0,7.8542
4,1.0,6.0,31.2750
...,...,...,...
707,0.0,21.0,7.6500
708,0.0,NaN,31.0000
709,1.0,41.0,14.1083
710,1.0,14.0,120.0000


In [33]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6312849162011173

In [34]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

np.float64(0.6554681647940075)